# Svenskt Kvinnobiografiskt lexikon 
version 1.3.4.12

* Denna [Jupityr Notebook](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon.ipynb) 
* Wikidataproperty [P4963](https://www.wikidata.org/wiki/Property_talk:P4963) 
  * Wikidata ung. 1200 kopplingar SKBL 7/04/2020
  * Wikidata 1449 SKBL 20/08/2020
* SKBL databas [KARP](https://ws.spraakbanken.gu.se/ws/karp)
  * 1495 poster 7/04/2020
  * 1512 poster 16/08/2020
  * 1517 poster 20/08/2020 (är fel pga dålig API fråga)
  
  Jag nås på 070-5937579 twitter: [salgo60](https://twitter.com/salgo60)
  
### TODO
* bättre API fråga så alla returneras skickat fråga GITHUB [issue 22](https://github.com/spraakbanken/skbl-portal/issues/22)
* prata ihop oss med SKBL hur vi kan enklare använda varandras data - får en känsla av att detta är ett projekt som snart avslutas dvs. för sent....
  * församlingar skrivs idag ut i text i JSON filen med koordinat vore tydligare om ni hade ett Qnummer med exempel
    * Borrby församling visas idag med text och 
    place: { parish: "Borrby församling", pin: { lat: 55.456944, lon: 14.180278 }.....
    * vore tydligare om ni hade med
        * unikt identifierade metadata som
          * sv:Wikipedia [Borrby_församling](https://sv.wikipedia.org/wiki/Borrby_f%C3%B6rsamling)  
          * Wikidata [Q10433305](https://www.wikidata.org/wiki/Q10433305?uselang=sv)
          * NAD nummer [SE/LLA/13038](https://sok.riksarkivet.se/?postid=ArkisRef%20SE/LLA/13038)
        * se [video](https://www.youtube.com/watch?v=qwVL0OOEdkY)
  * namnet blir idag lite grötigt med "/vera/" skapa hellre nya fält
  * kontakter anges idag som textsträngar bättre med samma som ISNI, Wikidata Qnummer eller annat så vi kan enklare vara säkra på rätt person
  * författare som exempel Brita Plank ange hennes Qnummer eller hennes ORCID
    * --> så kan vi enkelt ange rätt författare i Wikidata 
      * Brita Planck = [WD Q96205230](https://www.wikidata.org/wiki/Q96205230?uselang=sv) ORCID [0000-0002-1043-7455](https://orcid.org/0000-0002-1043-7455) WD [graf](https://w.wiki/ZxA)
      * Lisbeth Larsson [WD Q4961115](https://www.wikidata.org/wiki/Q4961115?uselang=sv) ORCID ''missing'' WD [graf](https://w.wiki/Zv8)
      * Marie Sjöberg [WD Q50383736](https://www.wikidata.org/wiki/Q50383736?uselang=sv) ORCID ''missing'' WD [graf](https://w.wiki/Zv7)

  
## Statistik
* idag länkar sv:Wikipedia se [video](https://www.youtube.com/watch?v=qwVL0OOEdkY)
  * med mall SKBL --> [456 artiklar](https://sv.wikipedia.org/wiki/Kategori:SKBL) --> 2019 antal visningar > [1 700 000](https://pageviews.toolforge.org/massviews/?platform=all-access&agent=user&source=category&range=this-year&subjectpage=0&subcategories=0&sort=views&direction=1&view=list&target=https://sv.wikipedia.org/wiki/Kategori:SKBL)
  * med mall Auktioritetsdata --> [1379 artiklar](https://sv.wikipedia.org/wiki/Kategori:Wikipediaartiklar_med_identifierare_fr%C3%A5n_SKBL) --> 2019 antal visningar > [3,4 miljoner](https://pageviews.toolforge.org/massviews/?platform=all-access&agent=user&source=category&range=this-year&subjectpage=0&subcategories=0&sort=views&direction=1&view=list&target=https://sv.wikipedia.org/wiki/Kategori:Wikipediaartiklar_med_identifierare_fr%25C3%25A5n_SKBL)
  * vi borde kunna lyfta detta vidare och ha samma som för orter/församlingar/organisationer/skolor
  
### sv:Wikipedia     
* SKBL artiklar som [saknar motsvarande artikel på sv:Wikipedia](https://w.wiki/Zui) 
* SKBL artiklar som [saknar Auktoritetsdatamallen](https://petscan.wmflabs.org/?psid=17145215)

### en:Wikipedia    
en:Wikipedia är en av världens 10 största siter så SKBLs engelska artiklar borde försöka länkas därifrån se [länk visningar](https://stats.wikimedia.org/v2/#/en.wikipedia.org)
* SKBL kopplade poster som [har en:Wikipedia artikel](https://w.wiki/ZxJ) = 680 stycken 
  * som [saknar en:Wikipedia artikel](https://w.wiki/Zup)
* ~~borde tas fram en [SKBL mall](https://sv.wikipedia.org/wiki/Mall:SKBL) på engelsk Wikipedia som pekar på SKBLs engelska artikel~~
  * [Q98518115](https://www.wikidata.org/wiki/Q98518115)  
* borde försöka få med SKBL i en:Wikipedia motsvarighet till Auktoritetsdata [Template:Authority_control](https://en.wikipedia.org/wiki/Template:Authority_control)  

#### SKBL template  live  på en:Wikipedia
Update its live se [video](https://youtu.be/a8Uty1etPdQ) 
* Mall SKBL finns nu på en:Wikipedia se [Template:SKBL](https://en.wikipedia.org/wiki/Template:SKBL) 
  * används på sidorna -> [Category:SKBL_template_using_Wikidata_property_P4963](https://en.wikipedia.org/wiki/Category:SKBL_template_using_Wikidata_property_P4963)
  * [visningsstatistik på dessa sidor](https://pageviews.toolforge.org/massviews/?platform=all-access&agent=user&source=category&range=this-year&subjectpage=0&subcategories=0&sort=views&direction=1&view=list&target=https://en.wikipedia.org/wiki/Category:SKBL_template_using_Wikidata_property_P4963)
* SKBL artiklar på en:Wikipedia som [saknar SKBL mallen](https://petscan.wmflabs.org/?psid=17136250)
  * idag finns 680 en:WIkipedia artiklar som i Wikidata har koppling SKBL se [Petscan fråga](https://petscan.wmflabs.org/?psid=17144813)
    * video hur [petscan sparql fungerar](https://youtu.be/hOIOc3SKKW4)  
    
### ru:Wikipedia
video om hur [Wikipedia och olika språkversioner fungerar](https://youtu.be/GzJfXsdmfeU)
ryska WIkipedia har varit rel enkla att lyfta in Wikidata egenskaper dom har idag [länk visningar](https://stats.wikimedia.org/v2/#/ru.wikipedia.org) 806 M visningar per månad så det kan vara ett spännande möte
* SKBL kopplade poster som [har ru:Wikipedia artikel](https://w.wiki/Zus) = 568 stycken samma men med [länk ru artikeln](https://w.wiki/Zuz)
  * som [saknar ru:Wikipedia artikel](https://w.wiki/Zup)
* Auktoritetsdata på ru:Wikipedia [Шаблон:Внешние ссылки](https://ru.wikipedia.org/wiki/%D0%A8%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD:%D0%92%D0%BD%D0%B5%D1%88%D0%BD%D0%B8%D0%B5_%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8)

## Chrome extension
Den chrome extension som utvecklas där man kan stå på en sida hos SKBL och slå upp i Wikidata [Entity Explosion](https://chrome.google.com/webstore/detail/entity-explosion/bbcffeclligkmfiocanodamdjclgejcn) 

# Check diff SKBL Wikidata 
ett försök att hålla Wikidata / SKBL i fas 
* [KARP APIet](https://spraakbanken.gu.se/verktyg/karp/webb-api)
* se tidigare försök att ha ISNI i synk [T219700](https://phabricator.wikimedia.org/T219700), GITHUB [salgo60/SKBLWikidata](https://github.com/salgo60/SKBLWikidata) 
  * mar 2019 kommentar om [platser i SKBL](https://phabricator.wikimedia.org/T218782)  
  
Viss osäkerhet hur en bra API fråga skall se ut returnerar alla personer i SKBL

In [244]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None)
pd.set_option('display.max_rows', None)

dftot = pd.DataFrame()
 
url = "https://ws.spraakbanken.gu.se/ws/karp/v5/minientry?q=extended%7C%7Cand%7Cnamn%7Cexists&mode=skbllinks&show=name%2Curl&sort=sorteringsnamn.sort%2Csorteringsnamn.init%2Ctilltalsnamn.sort&size=10000&resource=skbl"
r = http.request('GET', url) 
data = json.loads(r.data.decode('utf-8')) 
#print(type(data))
#print(r.info())
listSKBL = []
i = 0 
for row in (data["hits"]["hits"]):
    #print (row["_source"]["url"])
    listSKBL.append(row["_source"]["url"])
    i = i + 1 
    #print (i)
SKBLtot = pd.DataFrame(listSKBL,
                  columns=['skblid'])


In [245]:
SKBLtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1517 entries, 0 to 1516
Data columns (total 1 columns):
skblid    1517 non-null object
dtypes: object(1)
memory usage: 12.0+ KB


In [246]:
#SKBLtot

In [247]:
#SKBLtot.sort_index(inplace=True) 
SKBLtot.sort_values(['skblid'], ascending=[1],inplace=True) 

In [248]:
# The API gives us some duplicates 
SKBLtot.drop_duplicates(subset ="skblid", keep = False, inplace = True)
SKBLtot.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1388 entries, 538 to 817
Data columns (total 1 columns):
skblid    1388 non-null object
dtypes: object(1)
memory usage: 21.7+ KB


In [249]:
SKBLtot = SKBLtot.reset_index(drop=True)  
#SKBLtot

# Wikidata

In [250]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT * WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P4963 ?skbl
} order by ?skbl"""


def get_results(endpoint_url, query):
    user_agent = "User:salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)
 
lst = []
for result in results["results"]["bindings"]:
    lst.append((result["skbl"]["value"]))
WDtot = pd.DataFrame(lst,columns={'skblid'}) 
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1451 entries, 0 to 1450
Data columns (total 1 columns):
skblid    1451 non-null object
dtypes: object(1)
memory usage: 11.5+ KB


In [251]:
WDtot.sort_index(inplace=True)  

In [252]:
WDtot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1451 entries, 0 to 1450
Data columns (total 1 columns):
skblid    1451 non-null object
dtypes: object(1)
memory usage: 11.5+ KB


In [253]:
#WDtot

### Check duplicates SKBL and Wikidata

In [254]:
dfmerge = pd.merge(WDtot, SKBLtot,how='outer', indicator=True)

In [255]:
# check whatsnew / deleted  
dfmerge['_merge'].value_counts()

both          1388
left_only       63
right_only       0
Name: _merge, dtype: int64

In [256]:
#dfmerge

In [257]:
SKBLnew = dfmerge[dfmerge['_merge']=="right_only"]
SKBLnew_left = dfmerge[dfmerge['_merge']=="left_only"]


In [258]:
SKBLnew.columns

Index(['skblid', '_merge'], dtype='object')

In [259]:
SKBLnew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 2 columns):
skblid    0 non-null object
_merge    0 non-null category
dtypes: category(1), object(1)
memory usage: 104.0+ bytes


In [260]:
import csv  
jsonURL = "https://skbl.se/sv/artikel/"
listNewItems =[]
for index,row in SKBLnew.iterrows():
    url = jsonURL + row["skblid"] + ".json"
    r = http.request('GET', url) 
    data = json.loads(r.data.decode('utf-8'))  
    
    new_item = dict()
    try:
        birthdate = data['lifespan']['from']['date']['date'],
    except:
        birtdate =""
    new_item['birthdate'] = birthdate
    try:
        deathdate = data['lifespan']['to']['date']['date']
    except:
        deathdate=""
    new_item['deathdate'] = deathdate
    try:
        birthparish = data['lifespan']['from']['place']['parish']
    except:
        birthparish = ""        
    new_item['birthparish'] = birthparish
    try:
        deathparish = data['lifespan']['to']['place']['parish']
    except:
        deathparish = ""
    new_item['deathparish'] = deathparish
    try:
        ISNI = data['id']['id']
    except:
        ISNI = ""
    new_item['ISNI'] = ISNI
    new_item['SKBL'] = row["skblid"]
    new_item['index'] = index
    new_item['skbl_status'] = data['skbl_status']
    new_item['firstname'] = data['name'].get('firstname')
    new_item['lastname'] = data['name'].get('lastname')
    new_item['subtitle'] = data['subtitle']
    listNewItems.append(new_item)

if len(listNewItems):
    keys = listNewItems[0].keys()
with open("SKBL_newfile.csv", "w", newline='') as SKBLfile:
    dict_writer = csv.DictWriter(SKBLfile, keys)
    dict_writer.writeheader()
    dict_writer.writerows(listNewItems)

            
    

In [261]:
SKBLnew

,skblid,_merge


In [262]:
SKBLnew_left

,skblid,_merge
0,AaseNordmoLvberg,left_only
33,AliceBabsSjoblom,left_only
86,AnnaAlidaTheresiaNilsson,left_only
89,AnnaBaner,left_only
91,AnnaBielke,left_only
112,AnnaElisabethAlbertinaBergstrandPoulsen,left_only
133,AnnaLisaOstLappLisa,left_only
143,AnnaMargaretavonBragner,left_only
162,AnnaQNilsson,left_only
224,AugustaJansson0,left_only
